In [1]:
from pyspark.sql import SparkSession
import glob
import os

In [2]:
spark = SparkSession\
        .builder\
        .appName("simple s3 read")\
        .config("spark.jars.packages", "databricks:spark-deep-learning:1.5.0-spark2.4-s_2.11")\
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.3 pyspark-shell")\
        .getOrCreate()

In [3]:
import tensorflow as tf

/Users/derricklewis/anaconda3/envs/pipeline/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/derricklewis/anaconda3/envs/pipeline/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/derricklewis/anaconda3/envs/pipeline/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/derricklewis

In [16]:
from sparkdl import DeepImageFeaturizer

Using TensorFlow backend.
/Users/derricklewis/anaconda3/envs/pipeline/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/derricklewis/anaconda3/envs/pipeline/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/derricklewis/anaconda3/envs/pipeline/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16

In [17]:
from keras.applications import InceptionV3

# Load inception v3 (Best Image Classifier)
model = InceptionV3(weights="imagenet")

In [18]:
model.save('model-full.h5')

In [20]:
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.image import img_to_array, load_img
import numpy as np
import os
from pyspark.sql.types import StringType
from sparkdl import KerasImageFileTransformer

In [21]:
# Parameters
SIZE = (299, 299)  # Size accepted by Inception model
IMAGES_PATH = 'photos/'  # Images Path
MODEL = '/Users/derricklewis/Documents/Flatiron/Data-Pipeline-Capstone-Project/model-full.h5'  # Model Path

In [22]:
def preprocess_keras_inceptionV3(uri):
  image = img_to_array(load_img(uri, target_size=SIZE))
  image = np.expand_dims(image, axis=0)
  return preprocess_input(image)

In [23]:
transformer = KerasImageFileTransformer(inputCol="uri", outputCol="predictions",
                                        modelFile=MODEL,
                                        imageLoader=preprocess_keras_inceptionV3,
                                        outputMode="vector")

In [29]:
dirpath = '/Users/derricklewis/Documents/Flatiron/Data-Pipeline-Capstone-Project/photos/'

In [30]:
files = [os.path.abspath(os.path.join(dirpath, f)) for f in os.listdir(IMAGES_PATH) if f.endswith('.jpg')]
uri_df = sqlContext.createDataFrame(files, StringType()).toDF("uri")

In [31]:
uri_df.show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------+
|uri                                                                                                                       |
+--------------------------------------------------------------------------------------------------------------------------+
|/Users/derricklewis/Documents/Flatiron/Data-Pipeline-Capstone-Project/photos/fohrimage.jpg                                |
|/Users/derricklewis/Documents/Flatiron/Data-Pipeline-Capstone-Project/photos/open-uri20190201-31112-1kuo5ul?1548986925.jpg|
|/Users/derricklewis/Documents/Flatiron/Data-Pipeline-Capstone-Project/photos/fohrimage3.jpg                               |
|/Users/derricklewis/Documents/Flatiron/Data-Pipeline-Capstone-Project/photos/fohrimage2.jpg                               |
+--------------------------------------------------------------------------------------------------------------------------+


In [32]:
labels_df = transformer.transform(uri_df)

INFO:tensorflow:Froze 378 variables.
INFO:tensorflow:Converted 378 variables to const ops.

INFO:tensorflow:Froze 0 variables.
INFO:tensorflow:Converted 0 variables to const ops.




IllegalArgumentException: "NodeDef mentions attr 'index_type' not in Op<name=Fill; signature=dims:int32, value:T -> output:T; attr=T:type>; NodeDef: given/batch_normalization_94/ones_like = Fill[T=DT_FLOAT, index_type=DT_INT32](given/batch_normalization_94/ones_like/Shape, given/batch_normalization_94/ones_like/Const). (Check whether your GraphDef-interpreting binary is up to date with your GraphDef-generating binary.)."

In [ ]:
labels_df.show()

In [4]:
from __future__ import print_function
import boto3
# $example on$
import numpy as np
from math import sqrt
# $example off$

from pyspark import SparkContext
# $example on$
from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.sql.functions import udf

In [5]:
import configparser
config = configparser.ConfigParser()
config.read(os.path.expanduser("~/.aws/credentials"))
access_id = config.get('fohr_derrick', "aws_access_key_id") 
access_key = config.get('fohr_derrick', "aws_secret_access_key")

In [6]:
hadoop_conf=spark._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3native.NativeS3FileSystem")
hadoop_conf.set("fs.s3a.awsAccessKeyId", access_id)
hadoop_conf.set("fs.s3a.awsSecretAccessKey", access_key)

In [7]:
IMAGES_PATH = 's3a://social-system-test/instagram_graph_image_store/1/*/'

In [8]:
df = spark.read.format("image").option("dropInvalid", True).load(IMAGES_PATH)

In [9]:
df.show()

+--------------------+
|               image|
+--------------------+
|[s3a://social-sys...|
|[s3a://social-sys...|
|[s3a://social-sys...|
|[s3a://social-sys...|
|[s3a://social-sys...|
|[s3a://social-sys...|
|[s3a://social-sys...|
|[s3a://social-sys...|
|[s3a://social-sys...|
|[s3a://social-sys...|
|[s3a://social-sys...|
|[s3a://social-sys...|
|[s3a://social-sys...|
+--------------------+



In [10]:
df.select("image.data").show()

+--------------------+
|                data|
+--------------------+
|[BF CA CE BF CA C...|
|[BF CA CE BF CA C...|
|[FA F7 F3 FA F7 F...|
|[FA F7 F3 FA F7 F...|
|[FA F7 F3 FA F7 F...|
|[FA F7 F3 FA F7 F...|
|[CF CF CF CE CE C...|
|[CF CF CF CE CE C...|
|[C8 CA CA C8 CA C...|
|[C8 CA CA C8 CA C...|
|[F5 F0 ED F5 F0 E...|
|[F5 F0 ED F5 F0 E...|
|[FF FF FF FF FF F...|
+--------------------+



In [11]:
import cv2

In [28]:
def decode_image(url):
#     img_array = np.array(bytearray(url), dtype=np.uint8)
    return cv2.imdecode(url, 16)

In [29]:
get_img_array = udf(lambda x: decode_image(x))

In [30]:
df2 = df.withColumn("img_array", get_img_array(df.image.data))

In [31]:
df2.show()

Py4JJavaError: An error occurred while calling o247.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 14.0 failed 1 times, most recent failure: Lost task 0.0 in stage 14.0 (TID 30, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark-2.4.5/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/opt/spark-2.4.5/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/opt/spark-2.4.5/python/lib/pyspark.zip/pyspark/serializers.py", line 352, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "/opt/spark-2.4.5/python/lib/pyspark.zip/pyspark/serializers.py", line 142, in dump_stream
    for obj in iterator:
  File "/opt/spark-2.4.5/python/lib/pyspark.zip/pyspark/serializers.py", line 341, in _batched
    for item in iterator:
  